# 🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB

Objetivo: dominar la conexión, modelado básico, consultas eficientes e inserciones masivas en PostgreSQL (relacional) y MongoDB (NoSQL), con prácticas de índices, agregaciones y particionado.

- Duración: 90-120 min
- Dificultad: Media
- Prerrequisitos: Python y SQL básicos, nociones de modelado de datos

## Requisitos y entorno

- Este notebook incluye bloques opcionales para conectarte a PostgreSQL y MongoDB reales si tienes servicios disponibles.
- También incluye demostraciones auto-contenidas con SQLite para la parte SQL.
- Variables de entorno sugeridas: `POSTGRES_URI`, `MONGODB_URI`.
- Opcional: usa Docker para levantar servicios locales de Postgres y MongoDB.

### 🗄️ **SQL vs NoSQL: Paradigmas Complementarios**

**Comparación Fundamental:**

| Aspecto | PostgreSQL (Relacional) | MongoDB (Documento) |
|---------|-------------------------|---------------------|
| **Modelo** | Tablas + filas + columnas | Colecciones + documentos JSON (BSON) |
| **Schema** | Rígido (DDL: CREATE TABLE) | Flexible (schema-less) |
| **Relaciones** | Foreign Keys + JOINs | Embedding o References |
| **Transacciones** | ACID nativo (multi-tabla) | ACID desde v4.0 (multi-documento) |
| **Escalabilidad** | Vertical (scale-up) | Horizontal (sharding nativo) |
| **Query** | SQL estándar | MQL + Aggregation Pipeline |

**¿Cuándo usar cada uno?**

**PostgreSQL:**
- Datos estructurados con relaciones complejas (ERP, CRM)
- Transacciones críticas (finanzas, inventarios)
- Queries analíticos complejos con múltiples JOINs
- Cumplimiento regulatorio (auditoría, GDPR)

**MongoDB:**
- Datos semi-estructurados o variantes (logs, eventos)
- Alta frecuencia de escritura (IoT, telemetría)
- Necesidad de escalabilidad horizontal masiva
- Prototipado rápido sin schema fijo

**Patrón Híbrido (Polyglot Persistence):**
```
PostgreSQL (Transacciones) ←→ Kafka ←→ MongoDB (Analytics Real-Time)
                              ↓
                        Elasticsearch (Full-Text Search)
```

**SQLAlchemy como ORM Universal:**
- Abstracción sobre múltiples DBs (Postgres, MySQL, SQLite, Oracle)
- Connection pooling automático
- SQL expression language (type-safe vs raw SQL)

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. SQL con SQLAlchemy (demo con SQLite)

### 🐘 **PostgreSQL: SQL Avanzado para Analytics**

**Características Modernas de PostgreSQL:**

1. **JSONB (Binary JSON):**
   ```sql
   CREATE TABLE eventos (id SERIAL, payload JSONB);
   SELECT payload->>'user_id' FROM eventos WHERE payload @> '{"status":"active"}';
   ```
   - Indexable con GIN (Generalized Inverted Index)
   - Combina SQL + flexibilidad NoSQL

2. **Window Functions (Análisis Temporal):**
   ```sql
   SELECT 
     fecha,
     SUM(total) OVER (ORDER BY fecha ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as acumulado
   FROM ventas;
   ```
   - `LAG()`, `LEAD()`: Comparar con fila anterior/siguiente
   - `ROW_NUMBER()`, `RANK()`: Ranking con ties
   - `PARTITION BY`: Ventanas por grupo

3. **CTEs Recursivos (Common Table Expressions):**
   ```sql
   WITH RECURSIVE jerarquia AS (
     SELECT id, nombre, parent_id FROM categorias WHERE parent_id IS NULL
     UNION ALL
     SELECT c.id, c.nombre, c.parent_id FROM categorias c JOIN jerarquia h ON c.parent_id = h.id
   )
   SELECT * FROM jerarquia;
   ```

4. **Particionamiento (Table Partitioning):**
   ```sql
   CREATE TABLE ventas (fecha DATE, ...) PARTITION BY RANGE (fecha);
   CREATE TABLE ventas_2025 PARTITION OF ventas FOR VALUES FROM ('2025-01-01') TO ('2026-01-01');
   ```
   - **List**: Por valores discretos (regiones)
   - **Range**: Por rangos (fechas, IDs)
   - **Hash**: Distribución uniforme

**Índices Especializados:**
- **B-Tree** (default): Igualdad y rangos (`=`, `<`, `>`, `BETWEEN`)
- **Hash**: Solo igualdad exacta (más rápido que B-Tree)
- **GIN/GiST**: Full-text search, JSONB, arrays
- **BRIN**: Block Range Index para tablas enormes ordenadas

**EXPLAIN ANALYZE:**
```sql
EXPLAIN (ANALYZE, BUFFERS) SELECT ...;
```
- `Seq Scan`: Lectura secuencial (malo para tablas grandes)
- `Index Scan`: Usa índice (bueno)
- `cost=0.00..35.50`: Costo estimado vs real

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

# Usa PostgreSQL si POSTGRES_URI está disponible; de lo contrario, SQLite en memoria
pg_uri = os.getenv('POSTGRES_URI')
engine = create_engine(pg_uri) if pg_uri else create_engine('sqlite+pysqlite:///:memory:', echo=False, future=True)

with engine.begin() as conn:
    conn.execute(text('''
        CREATE TABLE IF NOT EXISTS ventas (
          id INTEGER PRIMARY KEY,
          fecha TEXT,
          cliente_id INTEGER,
          producto_id INTEGER,
          cantidad INTEGER,
          precio REAL,
          total REAL
        );
    '''))

df = pd.DataFrame({
    'id': [1,2,3,4],
    'fecha': ['2025-10-01','2025-10-02','2025-10-02','2025-10-03'],
    'cliente_id': [10,10,11,12],
    'producto_id': [101,102,101,103],
    'cantidad': [1,2,1,3],
    'precio': [100.0, 50.0, 100.0, 20.0]
})
df['total'] = df['cantidad'] * df['precio']
df.to_sql('ventas', engine, if_exists='append', index=False)

with engine.connect() as conn:
    res = conn.execute(text('SELECT cliente_id, SUM(total) as total FROM ventas GROUP BY cliente_id ORDER BY total DESC'))
    rows = res.fetchall()
rows

[(10, 200.0), (11, 100.0), (12, 60.0)]

### 1.1 Índices y optimización (PostgreSQL)

### 🔍 **Optimización SQL: Índices Compuestos**

**Estrategia de Índices:**

1. **Índice Simple vs Compuesto:**
   ```sql
   -- Simple (una columna)
   CREATE INDEX idx_cliente ON ventas (cliente_id);
   
   -- Compuesto (múltiples columnas - orden importa!)
   CREATE INDEX idx_cliente_fecha ON ventas (cliente_id, fecha);
   ```

2. **Regla de Selectividad (Left-to-Right):**
   - El índice `(cliente_id, fecha)` acelera:
     - ✅ `WHERE cliente_id = 10`
     - ✅ `WHERE cliente_id = 10 AND fecha > '2025-01-01'`
     - ❌ `WHERE fecha > '2025-01-01'` (no usa índice)
   
   - **Orden óptimo**: Columnas más selectivas primero
     - Selectividad alta: `email` (único)
     - Selectividad baja: `género` (M/F/O)

3. **Covering Index (Include Columns):**
   ```sql
   CREATE INDEX idx_covering ON ventas (cliente_id) INCLUDE (total);
   -- Query solo lee índice sin acceder a la tabla (Index-Only Scan)
   ```

4. **Índice Parcial (Filtered Index):**
   ```sql
   CREATE INDEX idx_activos ON usuarios (email) WHERE estado = 'activo';
   -- Más pequeño y rápido si solo consultas usuarios activos
   ```

**VACUUM y ANALYZE:**
- `VACUUM`: Libera espacio de filas eliminadas (bloat)
- `ANALYZE`: Actualiza estadísticas para el query planner
- `VACUUM ANALYZE`: Ambos en producción (mejor con autovacuum)

**Bloat y Reindex:**
```sql
-- Detectar bloat
SELECT schemaname, tablename, pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename))
FROM pg_tables WHERE schemaname NOT IN ('pg_catalog', 'information_schema');

-- Reindexar (libera espacio de índices fragmentados)
REINDEX INDEX idx_cliente_fecha;
```

**Trade-offs:**
- ✅ Índices: Aceleran SELECT/WHERE/JOIN
- ❌ Índices: Ralentizan INSERT/UPDATE/DELETE (mantenimiento)
- Regla: No más de 5-7 índices por tabla en OLTP

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [2]:
# Estas sentencias funcionan si usas PostgreSQL real
sql_index = '''
CREATE INDEX IF NOT EXISTS idx_ventas_cliente_fecha ON ventas (cliente_id, fecha);
ANALYZE ventas;
EXPLAIN ANALYZE SELECT * FROM ventas WHERE cliente_id = 10 AND fecha >= '2025-10-02';
'''
print(sql_index)


CREATE INDEX IF NOT EXISTS idx_ventas_cliente_fecha ON ventas (cliente_id, fecha);
ANALYZE ventas;
EXPLAIN ANALYZE SELECT * FROM ventas WHERE cliente_id = 10 AND fecha >= '2025-10-02';



## 2. MongoDB con PyMongo

### 🍃 **MongoDB: Modelado de Documentos**

**Documento BSON (Binary JSON):**
```json
{
  "_id": ObjectId("507f1f77bcf86cd799439011"),
  "cliente_id": 10,
  "fecha": ISODate("2025-10-01T00:00:00Z"),
  "items": [
    {"producto_id": 101, "cantidad": 2, "precio": 100.0},
    {"producto_id": 102, "cantidad": 1, "precio": 50.0}
  ],
  "total": 250.0,
  "direccion": {
    "calle": "Av. Principal",
    "ciudad": "Madrid"
  }
}
```

**Patrones de Modelado:**

1. **Embedding (Desnormalización):**
   - Relación 1:1 o 1:pocos
   - Datos leídos juntos frecuentemente
   - ✅ Ventaja: 1 query, sin JOINs
   - ❌ Desventaja: Límite de 16MB por documento

2. **Referencing (Normalización):**
   ```json
   // ventas collection
   {"_id": 1, "cliente_id": 10, "total": 250}
   
   // clientes collection
   {"_id": 10, "nombre": "Juan", "email": "juan@example.com"}
   ```
   - Relación 1:muchos con muchos items
   - ✅ Ventaja: Sin duplicación, updates centralizados
   - ❌ Desventaja: Múltiples queries o `$lookup` (JOIN)

3. **Patrón Subset:**
   - Embedar solo los N más recientes/relevantes
   ```json
   {
     "usuario_id": 10,
     "ultimas_compras": [...10 últimas...],  // Embedding
     "total_compras": 250                    // Referencia completa en otra collection
   }
   ```

**Índices en MongoDB:**
```javascript
db.ventas.createIndex({"cliente_id": 1, "fecha": -1})  // 1=ASC, -1=DESC
db.ventas.createIndex({"direccion.ciudad": 1})        // Nested field
db.ventas.createIndex({"items.producto_id": 1})       // Array indexing
db.ventas.createIndex({"total": 1}, {sparse: true})   // Solo docs con campo 'total'
```

**TTL Index (Auto-expiring):**
```javascript
db.logs.createIndex({"createdAt": 1}, {expireAfterSeconds: 2592000})  // 30 días
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

### 📦 Instalación de PyMongo

Antes de trabajar con MongoDB en Python, necesitas instalar la librería `pymongo`:

```bash
# Opción 1: pip directo
pip install pymongo

# Opción 2: Con versión específica
pip install pymongo==4.6.0

# Opción 3: Con extras para DNS SRV (MongoDB Atlas)
pip install pymongo[srv]
```

**Para MongoDB Atlas (Cloud):**
```bash
pip install "pymongo[srv]"
```

**Verificar instalación:**
```python
import pymongo
print(pymongo.version)
```

**Nota:** Si usas un entorno virtual, asegúrate de activarlo antes de instalar.

In [3]:
import os
from datetime import datetime

try:
    import pymongo
    MONGODB_URI = os.getenv('MONGODB_URI', 'mongodb://localhost:27017')
    # Descomenta para usar una base real
    # client = pymongo.MongoClient(MONGODB_URI)
    # db = client['demo']
    # col = db['ventas']
    # col.insert_many([
    #     {'_id':1,'fecha': datetime(2025,10,1), 'cliente_id':10, 'total':100.0},
    #     {'_id':2,'fecha': datetime(2025,10,2), 'cliente_id':10, 'total':100.0},
    #     {'_id':3,'fecha': datetime(2025,10,2), 'cliente_id':11, 'total':100.0},
    # ])
    # col.create_index([('cliente_id', 1), ('fecha', 1)])
    # pipeline = [
    #   {'$match': {'fecha': {'$gte': datetime(2025,10,2)}}},
    #   {'$group': {'_id': '$cliente_id', 'total': {'$sum': '$total'}}},
    #   {'$sort': {'total': -1}}
    # ]
    # list(col.aggregate(pipeline))
    print('PyMongo disponible; conecta si tienes un servidor MongoDB.')
except Exception as e:
    print('PyMongo no disponible o sin conexión:', e)

PyMongo disponible; conecta si tienes un servidor MongoDB.


### 🔄 **MongoDB Aggregation Pipeline**

**Stages del Pipeline:**

```javascript
db.ventas.aggregate([
  // 1. $match: Filtro (equivalente a WHERE)
  { $match: { fecha: { $gte: ISODate("2025-10-01") } } },
  
  // 2. $project: Selección/transformación de campos
  { $project: { 
      cliente_id: 1, 
      mes: { $month: "$fecha" },
      total: 1 
  }},
  
  // 3. $group: Agregación (equivalente a GROUP BY)
  { $group: { 
      _id: "$cliente_id", 
      total_ventas: { $sum: "$total" },
      promedio: { $avg: "$total" },
      count: { $sum: 1 }
  }},
  
  // 4. $sort: Ordenamiento
  { $sort: { total_ventas: -1 } },
  
  // 5. $limit: Top N
  { $limit: 10 },
  
  // 6. $lookup: JOIN con otra colección
  { $lookup: {
      from: "clientes",
      localField: "_id",
      foreignField: "cliente_id",
      as: "info_cliente"
  }},
  
  // 7. $unwind: Desagregar arrays
  { $unwind: "$items" },
  
  // 8. $out: Guardar resultado en nueva colección
  { $out: "top_clientes" }
])
```

**Optimización del Pipeline:**

1. **$match temprano**: Reducir docs antes de procesar
   ```javascript
   // ✅ Bueno (filtra primero)
   [{$match: {...}}, {$group: {...}}]
   
   // ❌ Malo (agrupa todo antes de filtrar)
   [{$group: {...}}, {$match: {...}}]
   ```

2. **Índices**: `$match` y `$sort` usan índices si están al inicio

3. **Allowskilldisk**: Para datasets grandes
   ```javascript
   db.ventas.aggregate([...], { allowDiskUse: true })
   ```

**Operadores Útiles:**
- **Acumuladores**: `$sum`, `$avg`, `$max`, `$min`, `$push`, `$addToSet`
- **Condicionales**: `$cond`, `$ifNull`, `$switch`
- **Strings**: `$concat`, `$substr`, `$toUpper`
- **Fechas**: `$dateToString`, `$year`, `$month`, `$dayOfWeek`

**Equivalencia SQL → MongoDB:**
```
SELECT cliente_id, SUM(total) as total_ventas
FROM ventas
WHERE fecha >= '2025-10-01'
GROUP BY cliente_id
HAVING SUM(total) > 1000
ORDER BY total_ventas DESC
LIMIT 10

↓↓↓

db.ventas.aggregate([
  {$match: {fecha: {$gte: ISODate("2025-10-01")}}},
  {$group: {_id: "$cliente_id", total_ventas: {$sum: "$total"}}},
  {$match: {total_ventas: {$gt: 1000}}},
  {$sort: {total_ventas: -1}},
  {$limit: 10}
])
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 3. Patrones de modelado y buenas prácticas

### 🛠️ **Patrones de Carga y Operaciones Masivas**

**PostgreSQL - Bulk Operations:**

1. **COPY (Más rápido para CSV):**
   ```sql
   COPY ventas FROM '/tmp/ventas.csv' WITH (FORMAT csv, HEADER true);
   -- 10-100x más rápido que INSERT individual
   ```

2. **Batch INSERT con VALUES:**
   ```python
   # Con SQLAlchemy
   conn.execute(
       text("INSERT INTO ventas (fecha, cliente_id, total) VALUES (:f, :c, :t)"),
       [{"f": "2025-10-01", "c": 10, "t": 100.0} for _ in range(10000)]
   )
   ```

3. **UPSERT (INSERT ... ON CONFLICT):**
   ```sql
   INSERT INTO ventas (id, total) VALUES (1, 100)
   ON CONFLICT (id) DO UPDATE SET total = EXCLUDED.total;
   ```
   - Idempotencia garantizada para reprocesamiento

**MongoDB - Bulk Operations:**

1. **insert_many() con ordered=False:**
   ```python
   col.insert_many(docs, ordered=False)
   # Continúa insertando incluso si algunos fallan
   ```

2. **bulk_write() (Control granular):**
   ```python
   from pymongo import InsertOne, UpdateOne, DeleteOne
   
   operations = [
       InsertOne({"_id": 1, "total": 100}),
       UpdateOne({"_id": 2}, {"$set": {"total": 200}}, upsert=True),
       DeleteOne({"_id": 3})
   ]
   col.bulk_write(operations, ordered=False)
   ```

3. **Update con upsert:**
   ```python
   col.update_one(
       {"_id": 1},
       {"$set": {"total": 100}, "$inc": {"count": 1}},
       upsert=True
   )
   ```

**Transaction Management:**

**PostgreSQL:**
```python
with engine.begin() as conn:  # Auto-commit/rollback
    conn.execute(stmt1)
    conn.execute(stmt2)
    # Si falla cualquiera → ROLLBACK automático
```

**MongoDB (Multi-Document Transactions):**
```python
with client.start_session() as session:
    with session.start_transaction():
        col1.insert_one({...}, session=session)
        col2.update_one({...}, session=session)
        # Si falla → rollback automático
```

**Best Practices:**
- Batch size óptimo: 500-1000 registros (balance memoria/latency)
- Connection pooling: Reutilizar conexiones (SQLAlchemy/PyMongo automático)
- Retry logic con exponential backoff para transient errors
- Monitoring: Duración de queries, conexiones activas, deadlocks

---
**Autor:** Luis J. Raigoso V. (LJRV)

- PostgreSQL: normalización moderada, claves surrogate, índices compuestos, particionamiento por rango/fecha.
- MongoDB: diseñar según patrones de acceso; embedding vs referencing; TTL e índices parciales.
- Carga: upserts idempotentes, lotes (batch size) y manejo de transacciones cuando aplique.
- Observabilidad: logging estructurado y métricas por operación.

## 4. Ejercicios

1. Diseña un índice compuesto para acelerar consultas por `producto_id` y `fecha`.
2. Escribe una consulta de ventas acumuladas por día (window function en PostgreSQL).
3. En MongoDB, crea un pipeline para obtener el top 3 de clientes por gasto en un rango de fechas.

---

## 🧭 Navegación

**← Anterior:** [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)

**Siguiente →:** [♻️ DataOps y CI/CD para Pipelines de Datos →](05_dataops_cicd.ipynb)

**📚 Índice de Nivel Mid:**
- [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb)
- [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb)
- [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb)
- [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer](03b_cloud_gcp.ipynb)
- [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)
- [🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB](04_bases_datos_postgresql_mongodb.ipynb) ← 🔵 Estás aquí
- [♻️ DataOps y CI/CD para Pipelines de Datos](05_dataops_cicd.ipynb)
- [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)
- [🧩 Optimización SQL y Particionado de Datos](07_optimizacion_sql_particionado.ipynb)
- [🚀 Servicios de Datos con FastAPI](08_fastapi_servicios_datos.ipynb)
- [🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación](09_proyecto_integrador_1.ipynb)
- [🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
